# Visual proprioception flow

Create the full flow for training models for visual proprioception. This notebook programmatically generates a set of exp/runs that cover all the necessary components for a visual proprioception system (sensor processing,  visual proprioception regressor and verification notebooks).

Then, it writes the exp/runs into an external directory full separated from the github source, and creates an automation script that runs them. A separate directory for the results is also created. 

Finally, it runs the necessary notebooks to execute the whole flow using papermill.

The results directory contain the output of this flow, both in terms of trained models, as well as results (in the verification exp/run).

In [ ]:
import sys
sys.path.append("..")
from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import pprint
import copy
import socket
import pathlib
import yaml
import tqdm
import papermill
# from automate import automate_exprun
import visproprio_helper 
from demonstration.demonstration import list_demos

# Setting up the separate directory
Setting up a separate directory for generated exp/run config files and the results. This cell will create a new directory. 

__Note__
Currently, as of 2025-09-03, we don't have a way to put the training data into this directory. Which means that once the code below creates the new directories, we need to manually copy the demonstrations as follows:
* the demonstration descriptor (eg. {demonstration_run}.yaml) into {directory_name}/exprun/demonstration
* the demonstration runs into {directory_name}/result/demonstration/{demonstration_run}/

It would be useful to set up a special import mechanism for this. 

In [ ]:
# the most likely changing names.
directory_name = "VisualProprioception_flow_00"
demonstration_cam = "dev0"
demonstration_run = "touch-apple"


hostname = socket.gethostname()
print(f"***Hostname is {hostname}")
if hostname == "raven":
    rootdir = pathlib.Path(f"~/WORK/_DataExternal/").expanduser()
elif hostname == "szenes.local":
    rootdir = pathlib.Path(f"~/Documents/Develop/Data/").expanduser()
elif hostname == "glassy":
    rootdir = pathlib.Path(f"~/Work/_DataExternal/").expanduser()
else:
    rootdir = pathlib.Path(Config()["experiment_external"])
exprun_path, result_path = visproprio_helper.external_setup(directory_name, rootdir)


In [ ]:
def generate_sensorprocessing_conv_vae(exprun_path, result_path, params, exp_name, run_name):
    """Generate the experiment for the training of the conv-vae sensorprocessing with the right training data and parameters. Returns a dictionary with the experiment, runname as well as an entry that will be used for the automation. 
    NOTE: a similar function is in Flow_BehaviorCloning.
    """
    val = {}
    val["latent_size"] = params["latent_size"]
    val["epochs"] = params["epochs"]
    val["save_period"] = 5
    val["training_data"] = params["training_data"]
    val["validation_data"] = params["validation_data"]
    # save the generated exprun spec
    path = pathlib.Path(Config().get_experiment_path(), exp_name, run_name + ".yaml")
    with open(path, "w") as f:
        yaml.dump(val, f)
    # now, generate the entry in the automation file 
    v = {}
    v["name"] = "Train_SP_Conv-VAE"
    v["notebook"] = "sensorprocessing/Train-Conv-VAE.ipynb"
    v["experiment"] = exp_name
    v["run"] = run_name
    v["external_path"] = exprun_path.as_posix()
    v["data_path"] = result_path.as_posix()
    return v

In [ ]:
def generate_sensorprocessing_propriotuned_cnn(exprun_path, result_path, params, exp_name, run_name):
    """Generate the experiment for the training of the propriotuned CNN with the right training data and parameters. 
    Returns a dictionary with the experiment, runname as well as an entry that will be used for the automation. 
    """
    val = copy.copy(params)
    # save the generated exprun spec
    path = pathlib.Path(Config().get_experiment_path(), exp_name, run_name + ".yaml")
    with open(path, "w") as f:
        yaml.dump(val, f)
    # now, generate the entry in the automation file 
    v = {}
    v["name"] = "Train_SP_Conv-VAE"
    v["notebook"] = "sensorprocessing/Train-Conv-VAE.ipynb"
    v["experiment"] = exp_name
    v["run"] = run_name
    v["external_path"] = exprun_path.as_posix()
    v["data_path"] = result_path.as_posix()
    return v

### Generate the exp/runs to be run

In [ ]:
expruns = []
# overall values
latent_size = 128
#
# Configuring the training and validation data, based 
# on all the demonstrations of a particular type
#
experiment = "demonstration"
exp = Config().get_experiment(experiment, demonstration_run)
demos = list_demos(exp)
print("***The demos considered")
pprint.pprint(demos)
# create a data structure from all the data directories 
# we considered
data = []
for demo in demos:
    data.append([demonstration_run, demo, demonstration_cam])
# the training data used to train/validate the sensor processor
sp_training_data = data[0:2]
sp_validation_data = data[2:4]
# the training data used to train/validate the visual proprioception regressor
vp_training_data = data[4:6]
vp_validation_data = data[6:]

# determine these values based on experience
epochs_sp = 300
epochs_bc = 500

# generate the vae exprun
exp_name = "sensorprocessing_conv_vae"
run_name = "sp_conv_vae_0001"
params = {}
params["latent_size"] = latent_size
params["epochs"] = epochs_sp
params["training_data"] = sp_training_data
params["validation_data"] = sp_validation_data
exprun = generate_sensorprocessing_conv_vae(
    exprun_path = exprun_path, result_path = result_path, params = params, exp_name = exp_name, run_name = run_name)
expruns.append(exprun)
